# Data Science en Entreprise

## API pôle emploi

Autorisation d’accès à l’api des offres d’emplois

Compte pole emploi :    
username : mohammed.didi@outlook.com    
password : Projet_Data1



In [6]:
import requests


def obtenir_token(client_id, client_secret):
    """
    Cette fonction obtient un token d'accès pour l'API de Pôle Emploi.
    """
    url_token = 'https://entreprise.pole-emploi.fr/connexion/oauth2/access_token?realm=/partenaire'
    auth_data = {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret,
        'scope': 'api_offresdemploiv2 o2dsoffre'
    }
    headers = {'Content-Type': 'application/x-www-form-urlencoded'}

    response = requests.post(url_token, data=auth_data, headers=headers)

    if response.status_code != 200:
        raise Exception(f"Erreur lors de la requête : {response.text}")
    else:
        data = response.json()
        return data['access_token']
    
# Utilisation de la fonction
client_id = 'PAR_projetdata_29c785675807663802eabdf57858fc4e2ecd4eba4de8d05eee293bfdcacefae5'
client_secret = '8a48c4726bb2d33cafb7a636703a3b0ad70e4098b525063d2a99387a95a76b70'


In [7]:
import requests
from datetime import datetime
import pandas as pd
import json


def rechercher_offres(token, mot_cle, mois, annee, nb_offres=10):
    """
    Recherche les offres d'emploi dans le domaine des RH pour un mois et une année spécifiques.
    """
    url_offres = 'https://api.emploi-store.fr/partenaire/offresdemploi/v2/offres/search'
    headers = {'Authorization': f'Bearer {token}'}

    # Calculer les dates de début et de fin du mois de novembre
    debut_mois = f"{annee}-{mois:02d}-01T00:00:00Z"
    fin_mois = f"{annee}-{mois:02d}-30T00:00:00Z"  # Novembre a 30 jours

    params = {
        'motsCles': mot_cle,  # Mots-clés pour le domaine des Ressources Humaines
        'range': f'0-{nb_offres-1}',  # Les 10 premières offres
        'minCreationDate': debut_mois,
        'maxCreationDate': fin_mois
    }

    response = requests.get(url_offres, headers=headers, params=params)

    try:
        return response.json()
    except:
        raise Exception(f"Erreur lors de la requête : {response.text}")



token = obtenir_token(client_id, client_secret)
offres_Data_novembre = rechercher_offres(token, 'Data', 11, 2023)  # Exemple pour novembre 2023
# json to dataframe
# jsoon to dataframe
df = pd.DataFrame.from_dict(offres_Data_novembre['resultats'])


In [8]:
df

,id,intitule,description,dateCreation,dateActualisation,lieuTravail,romeCode,romeLibelle,appellationlibelle,entreprise,...,origineOffre,offresManqueCandidats,formations,deplacementCode,deplacementLibelle,qualitesProfessionnelles,experienceCommentaire,langues,permis,agence
0,165JNGF,Consultant Data (H/F),Qui sommes-nous ?\n\nNous sommes passionnés pa...,2023-11-24T12:00:21.000Z,2024-01-04T17:11:52.000Z,"{'libelle': '13 - AIX EN PROVENCE', 'latitude'...",M1403,Études et prospectives socio-économiques,Data scientist,"{'nom': 'AMILTONE', 'logo': 'https://entrepris...",...,"{'origine': '1', 'urlOrigine': 'https://candid...",False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,165GXSJ,Ingénieur DATA-BI Talend (H/F),"LE CLIENT, votre futur employeur :\nLe siège s...",2023-11-23T10:57:11.000Z,2024-01-02T17:31:47.000Z,"{'libelle': '83 - TOULON', 'latitude': 43.1363...",M1805,Études et développement informatique,Développeur(se) décisionnel - Business Intelli...,"{'nom': 'OMNICIEL PROFILS ET SOLUTIONS', 'desc...",...,"{'origine': '1', 'urlOrigine': 'https://candid...",False,[{'niveauLibelle': 'Bac+5 et plus ou équivalen...,1,Jamais,"[{'libelle': 'Travailler en équipe', 'descript...",NaN,NaN,NaN,NaN
2,165GXPW,Ingénieur DATA-BI (H/F),"LE CLIENT, votre futur employeur :\nLe siège s...",2023-11-23T10:55:34.000Z,2024-01-03T18:23:20.000Z,"{'libelle': '13 - AIX EN PROVENCE', 'latitude'...",M1805,Études et développement informatique,Développeur(se) décisionnel - Business Intelli...,"{'nom': 'OMNICIEL PROFILS ET SOLUTIONS', 'desc...",...,"{'origine': '1', 'urlOrigine': 'https://candid...",False,[{'niveauLibelle': 'Bac+5 et plus ou équivalen...,1,Jamais,"[{'libelle': 'Travailler en équipe', 'descript...",ingénieur BI,NaN,NaN,NaN
3,165BDKT,Data Analyst expérimenté Business (H/F),Si tu es animé(e) par la passion de transforme...,2023-11-20T09:58:42.000Z,2024-01-02T10:15:17.000Z,"{'libelle': '84 - AVIGNON', 'latitude': 43.936...",M1403,Études et prospectives socio-économiques,Data analyst,"{'nom': 'AGENCE GLANUM', 'entrepriseAdaptee': ...",...,"{'origine': '1', 'urlOrigine': 'https://candid...",False,[{'niveauLibelle': 'Bac+5 et plus ou équivalen...,2,Ponctuels Zone nationale,"[{'libelle': 'Faire preuve de curiosité', 'des...",NaN,"[{'libelle': 'Anglais', 'exigence': 'E'}]",NaN,NaN
4,164YCGP,Délégué / Déléguée à la protection des données...,Vous serez amenés à vous déplacer sur la régio...,2023-11-16T15:16:51.000Z,2024-01-03T18:54:39.000Z,"{'libelle': '2B - PENTA DI CASINCA', 'latitude...",K1903,Défense et conseil juridique,Délégué(e) à la protection des données - DPO,"{'nom': 'Nota Bene', 'description': 'Notre str...",...,"{'origine': '1', 'urlOrigine': 'https://candid...",False,NaN,3,Fréquents Zone régionale,"[{'libelle': 'Faire preuve d'autonomie', 'desc...",NaN,NaN,"[{'libelle': 'B - Véhicule léger', 'exigence':...",NaN
5,164XVLK,Data analyst (H/F),Sous la responsabilité du chef de projet et en...,2023-11-16T12:27:08.000Z,2024-01-05T06:22:17.000Z,"{'libelle': '974 - LA POSSESSION', 'latitude':...",M1403,Études et prospectives socio-économiques,Data analyst,{'entrepriseAdaptee': False},...,"{'origine': '1', 'urlOrigine': 'https://candid...",False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'courriel': 'Pour postuler, utiliser le lien ..."
6,6965342,DBA Développeur SQL/Big Data MANTES LA VILLE ...,Dans le cadre dun important projet avec un gro...,2023-11-29T18:49:06.000Z,2023-11-29T18:49:06.000Z,"{'libelle': '78 - MANTES LA VILLE', 'latitude'...",M1805,Études et développement informatique,Développeur / Développeuse informatique,{'entrepriseAdaptee': False},...,"{'origine': '2', 'urlOrigine': 'https://candid...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,6964867,Coordinateur Litiges tarifaires et Master data...,Overview Coordinateur Litiges tarifaires et Ma...,2023-11-29T18:47:28.000Z,2023-11-29T18:47:28.000Z,"{'libelle': '92 - COLOMBES', 'latitude': 48.92...",M1802,Expertise et support en systèmes d'information,Data manager,{'entrepriseAdaptee': False},...,"{'origine': '2', 'urlO

In [9]:
df.columns

Index(['id', 'intitule', 'description', 'dateCreation', 'dateActualisation',
       'lieuTravail', 'romeCode', 'romeLibelle', 'appellationlibelle',
       'entreprise', 'typeContrat', 'typeContratLibelle', 'natureContrat',
       'experienceExige', 'experienceLibelle', 'competences', 'salaire',
       'dureeTravailLibelle', 'dureeTravailLibelleConverti', 'alternance',
       'contact', 'nombrePostes', 'accessibleTH', 'qualificationCode',
       'qualificationLibelle', 'codeNAF', 'secteurActivite',
       'secteurActiviteLibelle', 'origineOffre', 'offresManqueCandidats',
       'formations', 'deplacementCode', 'deplacementLibelle',
       'qualitesProfessionnelles', 'experienceCommentaire', 'langues',
       'permis', 'agence'],
      dtype='object')

In [10]:
# save to csv in ../data

df.to_csv('../data/offres_data_novembre.csv', index=False)
